# EEG - Flow

## 12. ICA apply EOG with no ICA 2 on EEG, this is to treat Fp1 and all as EOG
This steps requires that a comparison has been done between two reviewers.
Both person will either select the best selection or make a new one together, then will rename the file to match the step 5 convention.

Last edit: 21.06.2023 19:25

In [ ]:
import multiprocessing as mp
import time
from itertools import product

from mne import set_log_level as set_log_level_mne

from eeg_flow import set_log_level
from eeg_flow.utils import parallel
from eeg_flow.tasks import apply_ica_reref_EOG_final_extend

In [ ]:
set_log_level_mne("WARNING")
set_log_level("INFO")

The parameters of the file to process are defined below. Locks are created to prevent someone else from running the same task and from writing the same derivatives.

In [ ]:
PARTICIPANTS_WITH_GROUPS: list[str] = ["P17-G5", "P23-G7", "P43-G4", "P04-G3", "P07-G5",
  "P11-G1", "P37-G2", "P30-G4", "P59-G8", "P01-G1",
  "P22-G7", "P13-G3", "P06-G4", "P71-G8", 
  "P05-G2", "P26-G2", "P51-G3", "P64-G3", "P15-G4",
  "P18-G6", "P19-G6", "P28-G1", "P38-G8", "P67-G3",
  "P21-G7", "P24-G8", "P47-G7", "P58-G5", "P46-G3"]   # list of str "Pxx-Gy", e.g. ["P02-G2"]

PARTICIPANTS_WITH_GROUPS: list[str] = ["P07-G5", "P11-G1", "P13-G3", "P22-G7", "P23-G7", 
                                       "P30-G4", "P37-G2", "P43-G4", "P59-G8", "P71-G8",
                                     "P15-G4", "P18-G6", "P24-G8", "P38-G8", "P46-G3", 
                                       "P47-G7", "P51-G3", "P58-G5", "P67-G3"
]   # list of str "Pxx-Gy", e.g. ["P02-G2"]


PARTICIPANTS_WITH_GROUPS: list[str] = ["P07-G5", "P11-G1", "P13-G3", "P22-G7", "P23-G7", 
                                       "P30-G4", "P37-G2", "P43-G4", "P59-G8",
                                       "P15-G4", "P18-G6", "P24-G8", "P38-G8", "P46-G3", 
                                       "P47-G7", "P51-G3", "P58-G5", "P67-G3"
]   # list of str "Pxx-Gy", e.g. ["P02-G2"]

PARTICIPANTS_WITH_GROUPS: list[str] = ["P07-G5"
]   # list of str "Pxx-Gy", e.g. ["P02-G2"]

PARTICIPANTS_WITH_GROUPS: list[str] = ["P11-G1", "P13-G3", "P22-G7", "P23-G7", 
                                       "P30-G4", "P37-G2", "P43-G4", "P59-G8",
                                       "P15-G4", "P18-G6", "P24-G8", "P38-G8", "P46-G3", 
                                       "P47-G7", "P51-G3", "P58-G5", "P67-G3"
]   # list of str "Pxx-Gy", e.g. ["P02-G2"]

TASKS: list[str]                    = ["oddball","UT"]  # ["oddball"], ["UT"] or ["oddball", "UT"]
RUNS: list[int]                     = [1,2]  # [1], [2] or [1, 2]


inputs = [
    (t[0].split("-") + list(t[1:]))
    for t in product(PARTICIPANTS_WITH_GROUPS, TASKS, RUNS)
]
print(inputs)

The variable `inputs` contains is a list of list. Each sublist defines one file by its participant, group, task and run attribute. Each sublist is one set of input variable for `apply_ica` which will be picked up by a worker (process) and executed. For each execution, the created derivatives are:
- Preprocessed raw recording (`_step7_preprocessed_raw.fif`)

This step ends with a visual inspection of the raw recording during which `BAD` segments annotations should be edited based on what was well corrected and what was not.

In [ ]:
%%time
current_time = time.strftime("%H:%M:%S")
print("Start time", current_time)

assert len(inputs) != 0  # sanity-check
# each job will need 1 threads and mp.cpu_count() usually returns
# the number of threads (usually 2 per core), thus let's spawn 2
# process per core maximum.
n_jobs = min(len(inputs), (mp.cpu_count() - 2))
parallel(apply_ica_reref_EOG_final_extend, n_jobs, inputs)